# **Modelo de aprendizaje automático entrenado con elementos textuales, emojis y emoticones para la detección de mensajes de ciberacoso en español peruano en la red social X**
#### **Estudiante:** Nicole Maria Morales Robladillo

In [ ]:
!pip install emot
!pip install emoji
!pip install pyspellchecker
!pip install -U deep-translator
!pip install spacy -q
!pip install gensim -q
!python -m spacy download es_core_news_sm -q
!apt-get update
!apt-get install python-dev
!apt-get install -y hunspell-es
!apt-get install libhunspell-dev
!pip install hunspell

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 102.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ In

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 535, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 535 (delta 174), reused 130 (delta 94), pack-reused 269 (from 1)
Receiving objects: 100% (535/535), 172.47 KiB | 2.27 MiB/s, done.
Resolving deltas: 100% (276/276), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.3 MB/s eta 0:00:00
Installing RAPIDS remaining 24.10.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.7/567.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 935.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.5/915.5 kB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install emoji==2.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.2/553.2 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: emoji
    Found existing installation: emoji 2.14.0
    Uninstalling emoji-2.14.0:
      Successfully uninstalled emoji-2.14.0


In [ ]:
import cudf
import cuml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import emoji
import emot
import spacy
import nltk
import seaborn as sns
import gensim
import dask_cudf
import dask.dataframe as dd
import joblib
import hunspell
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import metrics
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from deep_translator import GoogleTranslator
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from cuml.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import drive
from cuml.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from cuml.naive_bayes import MultinomialNB
from cuml.svm import SVC
from cuml.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from cuml.neighbors import KNeighborsClassifier
from cuml.multiclass import MulticlassClassifier
from cuml.multiclass import OneVsOneClassifier
from cuml.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
import random
from nltk.corpus import wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## **2. Preprocesamiento**

In [ ]:
def obtenerDataframe(file_id):
  url_file = f'https://drive.google.com/uc?export=download&id={file_id}&confirm=t&uuid=8a9582f7-6557-4ee6-bbb1-95c1df86ab4f&at=AKKF8vxzwBgpKweKVgTEzt9GaIwm:1682688519751'
  df = pd.read_csv(url_file, sep=',', encoding='utf-8')
  df = df.dropna()
  df = df.reset_index(drop=True)
  df['tipo'] = df['tipo'].astype(int)
  return df[['tweet', 'tipo']]

In [ ]:
df = obtenerDataframe('1fSkKRXXZqFh_BK4UJ1RKGdP7OAhYJXX_')
df

,tweet,tipo
0,motosierra entonces abajo durazno uf,3
1,alana que ganas de tenerte así cara babeante ...,3
2,uy qué rico bv fuego durazno cara caliente,3
3,mujerzuela venus cuando no sabemos qué está d...,3
4,encabellar no sólo tu cabello también tu manza...,3
...,...,...
2391,manutención,0
2392,tu primera vez ojos esta fue mi primera colabo...,0
2393,universalidad hermoso aunque le falta un poco ...,0
2394,ayo las chilenas que se lograron encontrar con...,0


### **2.13. Remoción de los caracteres repetitivos**

In [ ]:
def eliminarCaracteresRepetitivos(texto):
  palabras_permitidas = {'rr', 'll', 'cc', 'ee', 'oo'}
  patron = re.compile(r'(\w)\1+')
  def reemplazo(match):
    char = match.group(1)
    repetido = match.group(0)
    if char * 2 in palabras_permitidas:
      if len(repetido) > 2:
        return char * 2
      else:
        return repetido
    else:
      return char
  texto_modificado = patron.sub(reemplazo, texto)
  return texto_modificado

In [ ]:
df.tweet = df.tweet.apply(eliminarCaracteresRepetitivos)
df

,tweet,tipo
0,motosierra entonces abajo durazno uf,3
1,alana que ganas de tenerte así cara babeante ...,3
2,uy qué rico bv fuego durazno cara caliente,3
3,mujerzuela venus cuando no sabemos qué está d...,3
4,encabellar no sólo tu cabello también tu manza...,3
...,...,...
2391,manutención,0
2392,tu primera vez ojos esta fue mi primera colabo...,0
2393,universalidad hermoso aunque le falta un poco ...,0
2394,ayo las chilenas que se lograron encontrar con...,0


### **2.14. Tokenización**

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def tokenizar(text):
  tokens = word_tokenize(text, language='spanish')
  return [word for word in tokens]

In [ ]:
df['tweet_tokenized'] = ''
df.tweet_tokenized = df.tweet.apply(tokenizar)
df

,tweet,tipo,tweet_tokenized
0,motosierra entonces abajo durazno uf,3,"[motosierra, entonces, abajo, durazno, uf]"
1,alana que ganas de tenerte así cara babeante ...,3,"[alana, que, ganas, de, tenerte, así, cara, ba..."
2,uy qué rico bv fuego durazno cara caliente,3,"[uy, qué, rico, bv, fuego, durazno, cara, cali..."
3,mujerzuela venus cuando no sabemos qué está d...,3,"[mujerzuela, venus, cuando, no, sabemos, qué, ..."
4,encabellar no sólo tu cabello también tu manza...,3,"[encabellar, no, sólo, tu, cabello, también, t..."
...,...,...,...
2391,manutención,0,[manutención]
2392,tu primera vez ojos esta fue mi primera colabo...,0,"[tu, primera, vez, ojos, esta, fue, mi, primer..."
2393,universalidad hermoso aunque le falta un poco ...,0,"[universalidad, hermoso, aunque, le, falta, un..."
2394,ayo las chilenas que se lograron encontrar con...,0,"[ayo, las, chilenas, que, se, lograron, encont..."


### **2.15. Reemplazar cada jerga por su sinónimo en español estándar**

In [ ]:
spanish_peruvian_dict_id = '17J9JgVzdSeXKMk6hhsA8Zv2XCjIdMH84'
spanish_peruvian_dict_url = f'https://drive.google.com/uc?export=download&id={spanish_peruvian_dict_id}&confirm=t&uuid=8a9582f7-6557-4ee6-bbb1-95c1df86ab4f&at=AKKF8vxzwBgpKweKVgTEzt9GaIwm:1682688519751'
spanish_peruvian_dict_df = pd.read_excel(spanish_peruvian_dict_url)
spanish_peruvian_dict_df = spanish_peruvian_dict_df.dropna()
spanish_peruvian_dict_df = spanish_peruvian_dict_df.reset_index(drop=True)
spanish_peruvian_dict_df

,jerga,significado
0,achorar,desafiar
1,afanar,elogiar
2,agarre,pareja
3,agarrar,besar
4,agarrado,fuerte
...,...,...
178,yapa,extra
179,ya pe,ya pues
180,yuca,difícil
181,yunaites,estados unidos


In [ ]:
spanish_peruvian_dict = dict(zip(spanish_peruvian_dict_df['jerga'], spanish_peruvian_dict_df['significado']))
def traducirJergas(tweet_tokenized):
  return [spanish_peruvian_dict.get(word, word) for word in tweet_tokenized]

In [ ]:
df['tweet_tokenized'] = df.tweet_tokenized.apply(traducirJergas)
df

,tweet,tipo,tweet_tokenized
0,motosierra entonces abajo durazno uf,3,"[motosierra, entonces, abajo, durazno, uf]"
1,alana que ganas de tenerte así cara babeante ...,3,"[alana, que, ganas, de, tenerte, así, cara, ba..."
2,uy qué rico bv fuego durazno cara caliente,3,"[uy, qué, rico, bv, fuego, durazno, cara, cali..."
3,mujerzuela venus cuando no sabemos qué está d...,3,"[mujerzuela, venus, cuando, no, sabemos, qué, ..."
4,encabellar no sólo tu cabello también tu manza...,3,"[encabellar, no, sólo, tu, cabello, también, t..."
...,...,...,...
2391,manutención,0,[manutención]
2392,tu primera vez ojos esta fue mi primera colabo...,0,"[tu, primera, vez, ojos, esta, fue, mi, primer..."
2393,universalidad hermoso aunque le falta un poco ...,0,"[universalidad, hermoso, aunque, le, falta, un..."
2394,ayo las chilenas que se lograron encontrar con...,0,"[ayo, las, chilenas, que, se, lograron, encont..."


### **2.16. Remoción de palabras no informativas**

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords_es = set(stopwords.words('spanish'))
def eliminarStopWords(tweet_tokenized):
  return [word for word in tweet_tokenized if word.lower() not in stopwords_es]

In [ ]:
df['tweet_tokenized'] = df.tweet_tokenized.apply(eliminarStopWords)
df

,tweet,tipo,tweet_tokenized
0,motosierra entonces abajo durazno uf,3,"[motosierra, entonces, abajo, durazno, uf]"
1,alana que ganas de tenerte así cara babeante ...,3,"[alana, ganas, tenerte, así, cara, babeante, d..."
2,uy qué rico bv fuego durazno cara caliente,3,"[uy, rico, bv, fuego, durazno, cara, caliente]"
3,mujerzuela venus cuando no sabemos qué está d...,3,"[mujerzuela, venus, sabemos, dentro, cama, ech..."
4,encabellar no sólo tu cabello también tu manza...,3,"[encabellar, sólo, cabello, manzana, durazno, ..."
...,...,...,...
2391,manutención,0,[manutención]
2392,tu primera vez ojos esta fue mi primera colabo...,0,"[primera, vez, ojos, primera, colaboración, co..."
2393,universalidad hermoso aunque le falta un poco ...,0,"[universalidad, hermoso, aunque, falta, mas, m..."
2394,ayo las chilenas que se lograron encontrar con...,0,"[ayo, chilenas, lograron, encontrar, juncáceo,..."


### **2.17. Análisis semántico**

#### **2.17.1. Lematización**

In [ ]:
nlp = spacy.load('es_core_news_sm')
def obtenerLemas(tweet_tokenized):
  return [nlp(word)[0].lemma_ for word in tweet_tokenized]

In [ ]:
df['tweet_tokenized_lematizacion'] = ''
df.tweet_tokenized_lematizacion = df.tweet_tokenized.apply(obtenerLemas)
df

,tweet,tipo,tweet_tokenized,tweet_tokenized_lematizacion
0,motosierra entonces abajo durazno uf,3,"[motosierra, entonces, abajo, durazno, uf]","[motosierra, entonces, abajo, durazno, uf]"
1,alana que ganas de tenerte así cara babeante ...,3,"[alana, ganas, tenerte, así, cara, babeante, d...","[alán, gana, tenerte, así, cara, babeante, dur..."
2,uy qué rico bv fuego durazno cara caliente,3,"[uy, rico, bv, fuego, durazno, cara, caliente]","[uy, rico, bv, fuego, durazno, cara, caliente]"
3,mujerzuela venus cuando no sabemos qué está d...,3,"[mujerzuela, venus, sabemos, dentro, cama, ech...","[mujerzuelar, venus, saber, dentro, cama, echa..."
4,encabellar no sólo tu cabello también tu manza...,3,"[encabellar, sólo, cabello, manzana, durazno, ...","[encabellar, sólo, cabello, manzana, durazno, ..."
...,...,...,...,...
2391,manutención,0,[manutención],[manutención]
2392,tu primera vez ojos esta fue mi primera colabo...,0,"[primera, vez, ojos, primera, colaboración, co...","[primero, vez, ojo, primero, colaboración, con..."
2393,universalidad hermoso aunque le falta un poco ...,0,"[universalidad, hermoso, aunque, falta, mas, m...","[universalidad, hermoso, aunque, falta, mas, m..."
2394,ayo las chilenas que se lograron encontrar con...,0,"[ayo, chilenas, lograron, encontrar, juncáceo,...","[ayo, chileno, lograr, encontrar, juncáceo, li..."


In [ ]:
def unirTokens(tweet):
  return ' '.join(tweet)

In [ ]:
df['tweet_tokenized_lematizacion_cleaned'] = ''
df['tweet_tokenized_lematizacion_cleaned'] = df.tweet_tokenized_lematizacion.apply(unirTokens).astype(str)
df

,tweet,tipo,tweet_tokenized,tweet_tokenized_lematizacion,tweet_tokenized_lematizacion_cleaned
0,motosierra entonces abajo durazno uf,3,"[motosierra, entonces, abajo, durazno, uf]","[motosierra, entonces, abajo, durazno, uf]",motosierra entonces abajo durazno uf
1,alana que ganas de tenerte así cara babeante ...,3,"[alana, ganas, tenerte, así, cara, babeante, d...","[alán, gana, tenerte, así, cara, babeante, dur...",alán gana tenerte así cara babeante durazno
2,uy qué rico bv fuego durazno cara caliente,3,"[uy, rico, bv, fuego, durazno, cara, caliente]","[uy, rico, bv, fuego, durazno, cara, caliente]",uy rico bv fuego durazno cara caliente
3,mujerzuela venus cuando no sabemos qué está d...,3,"[mujerzuela, venus, sabemos, dentro, cama, ech...","[mujerzuelar, venus, saber, dentro, cama, echa...",mujerzuelar venus saber dentro cama echar encima
4,encabellar no sólo tu cabello también tu manza...,3,"[encabellar, sólo, cabello, manzana, durazno, ...","[encabellar, sólo, cabello, manzana, durazno, ...",encabellar sólo cabello manzana durazno cara l...
...,...,...,...,...,...
2391,manutención,0,[manutención],[manutención],manutención
2392,tu primera vez ojos esta fue mi primera colabo...,0,"[primera, vez, ojos, primera, colaboración, co...","[primero, vez, ojo, primero, colaboración, con...",primero vez ojo primero colaboración conocí km...
2393,universalidad hermoso aunque le falta un poco ...,0,"[universalidad, hermoso, aunque, falta, mas, m...","[universalidad, hermoso, aunque, falta, mas, m...",universalidad hermoso aunque falta mas movimie...
2394,ayo las chilenas que se lograron encontrar con...,0,"[ayo, chilenas, lograron, encontrar, juncáceo,...","[ayo, chileno, lograr, encontrar, juncáceo, li...",ayo chileno lograr encontrar juncáceo lindo ca...


#### **2.17.2. Tallado**

In [ ]:
stemmer = SnowballStemmer('spanish')
def obtenerTallado(tweet_tokenized):
  return [stemmer.stem(word) for word in tweet_tokenized]

In [ ]:
df['tweet_tokenized_tallado'] = ''
df.tweet_tokenized_tallado = df.tweet_tokenized.apply(obtenerTallado)
df

,tweet,tipo,tweet_tokenized,tweet_tokenized_lematizacion,tweet_tokenized_lematizacion_cleaned,tweet_tokenized_tallado
0,motosierra entonces abajo durazno uf,3,"[motosierra, entonces, abajo, durazno, uf]","[motosierra, entonces, abajo, durazno, uf]",motosierra entonces abajo durazno uf,"[motosierr, entonc, abaj, durazn, uf]"
1,alana que ganas de tenerte así cara babeante ...,3,"[alana, ganas, tenerte, así, cara, babeante, d...","[alán, gana, tenerte, así, cara, babeante, dur...",alán gana tenerte así cara babeante durazno,"[alan, gan, tenert, asi, car, babeant, durazn]"
2,uy qué rico bv fuego durazno cara caliente,3,"[uy, rico, bv, fuego, durazno, cara, caliente]","[uy, rico, bv, fuego, durazno, cara, caliente]",uy rico bv fuego durazno cara caliente,"[uy, ric, bv, fueg, durazn, car, calient]"
3,mujerzuela venus cuando no sabemos qué está d...,3,"[mujerzuela, venus, sabemos, dentro, cama, ech...","[mujerzuelar, venus, saber, dentro, cama, echa...",mujerzuelar venus saber dentro cama echar encima,"[mujerzuel, venus, sab, dentr, cam, echam, encim]"
4,encabellar no sólo tu cabello también tu manza...,3,"[encabellar, sólo, cabello, manzana, durazno, ...","[encabellar, sólo, cabello, manzana, durazno, ...",encabellar sólo cabello manzana durazno cara l...,"[encabell, sol, cabell, manzan, durazn, car, l..."
...,...,...,...,...,...,...
2391,manutención,0,[manutención],[manutención],manutención,[manutencion]
2392,tu primera vez ojos esta fue mi primera colabo...,0,"[primera, vez, ojos, primera, colaboración, co...","[primero, vez, ojo, primero, colaboración, con...",primero vez ojo primero colaboración conocí km...,"[primer, vez, ojos, primer, colabor, conoc, km..."
2393,universalidad hermoso aunque le falta un poco ...,0,"[universalidad, hermoso, aunque, falta, mas, m...","[universalidad, hermoso, aunque, falta, mas, m...",universalidad hermoso aunque falta mas movimie...,"[universal, hermos, aunqu, falt, mas, movimien..."
2394,ayo las chilenas que se lograron encontrar con...,0,"[ayo, chilenas, lograron, encontrar, juncáceo,...","[ayo, chileno, lograr, encontrar, juncáceo, li...",ayo chileno lograr encontrar juncáceo lindo ca...,"[ayo, chilen, logr, encontr, juncace, lind, ca..."


In [ ]:
df['tweet_tokenized_tallado_cleaned'] = ''
df['tweet_tokenized_tallado_cleaned'] = df.tweet_tokenized_tallado.apply(unirTokens).astype(str)
df

,tweet,tipo,tweet_tokenized,tweet_tokenized_lematizacion,tweet_tokenized_lematizacion_cleaned,tweet_tokenized_tallado,tweet_tokenized_tallado_cleaned
0,motosierra entonces abajo durazno uf,3,"[motosierra, entonces, abajo, durazno, uf]","[motosierra, entonces, abajo, durazno, uf]",motosierra entonces abajo durazno uf,"[motosierr, entonc, abaj, durazn, uf]",motosierr entonc abaj durazn uf
1,alana que ganas de tenerte así cara babeante ...,3,"[alana, ganas, tenerte, así, cara, babeante, d...","[alán, gana, tenerte, así, cara, babeante, dur...",alán gana tenerte así cara babeante durazno,"[alan, gan, tenert, asi, car, babeant, durazn]",alan gan tenert asi car babeant durazn
2,uy qué rico bv fuego durazno cara caliente,3,"[uy, rico, bv, fuego, durazno, cara, caliente]","[uy, rico, bv, fuego, durazno, cara, caliente]",uy rico bv fuego durazno cara caliente,"[uy, ric, bv, fueg, durazn, car, calient]",uy ric bv fueg durazn car calient
3,mujerzuela venus cuando no sabemos qué está d...,3,"[mujerzuela, venus, sabemos, dentro, cama, ech...","[mujerzuelar, venus, saber, dentro, cama, echa...",mujerzuelar venus saber dentro cama echar encima,"[mujerzuel, venus, sab, dentr, cam, echam, encim]",mujerzuel venus sab dentr cam echam encim
4,encabellar no sólo tu cabello también tu manza...,3,"[encabellar, sólo, cabello, manzana, durazno, ...","[encabellar, sólo, cabello, manzana, durazno, ...",encabellar sólo cabello manzana durazno cara l...,"[encabell, sol, cabell, manzan, durazn, car, l...",encabell sol cabell manzan durazn car lengu ca...
...,...,...,...,...,...,...,...
2391,manutención,0,[manutención],[manutención],manutención,[manutencion],manutencion
2392,tu primera vez ojos esta fue mi primera colabo...,0,"[primera, vez, ojos, primera, colaboración, co...","[primero, vez, ojo, primero, colaboración, con...",primero vez ojo primero colaboración conocí km...,"[primer, vez, ojos, primer, colabor, conoc, km...",primer vez ojos primer colabor conoc km venezu...
2393,universalidad hermoso aunque le falta un poco ...,0,"[universalidad, hermoso, aunque, falta, mas, m...","[universalidad, hermoso, aunque, falta, mas, m...",universalidad hermoso aunque falta mas movimie...,"[universal, hermos, aunqu, falt, mas, movimien...",universal hermos aunqu falt mas movimient cole...
2394,ayo las chilenas que se lograron encontrar con...,0,"[ayo, chilenas, lograron, encontrar, juncáceo,...","[ayo, chileno, lograr, encontrar, juncáceo, li...",ayo chileno lograr encontrar juncáceo lindo ca...,"[ayo, chilen, logr, encontr, juncace, lind, ca...",ayo chilen logr encontr juncace lind car conte...


## **3. División de data de entrenamiento y prueba**

In [ ]:
X = df.drop('tipo', axis = 1)
y = df['tipo'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 101)

In [ ]:
y_train = y_train.values
y_test = y_test.values

## **4. Selección del modelo**

### **4.1. Con Lematización**

#### **4.1.1. TF-IDF**

##### **4.1.1.1. Naive Bayes Multinomial**

In [ ]:
pipeline_naive_bayes_clasificador_tf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

param_grid_naive_bayes_clasificador_tf = {
    'tfidf__max_df': [0.75, 1.0],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__alpha': [0.1, 0.5, 1.0, 5.0, 10.0],
    'clf__fit_prior': [True, False]
}

naive_bayes_clasificador_tf = GridSearchCV(estimator=pipeline_naive_bayes_clasificador_tf, param_grid=param_grid_naive_bayes_clasificador_tf, cv=5, n_jobs=-1, verbose=6, scoring='accuracy')

In [ ]:
naive_bayes_clasificador_tf_lematizacion = naive_bayes_clasificador_tf

naive_bayes_clasificador_tf_lematizacion.fit(X_train['tweet_tokenized_lematizacion_cleaned'].values, y_train)

y_pred_naive_bayes_clasificador_tf_lematizacion = naive_bayes_clasificador_tf_lematizacion.predict(X_test['tweet_tokenized_lematizacion_cleaned'].values)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


In [ ]:
joblib.dump(naive_bayes_clasificador_tf_lematizacion, 'naive_bayes_clasificador_tf_lematizacion.pkl')

['naive_bayes_clasificador_tf_lematizacion.pkl']

##### **4.1.1.2. Máquina de Vectores de Soporte**

In [ ]:
pipeline_maquina_vectores_soporte_clasificador_tf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', SVC())
])

param_grid_maquina_vectores_soporte_clasificador_tf = {
    'tfidf__max_df': [0.75, 1.0],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__C': [0.1, 1, 10, 100],
    'clf__gamma': ['scale', 'auto'],
    'clf__kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

maquina_vectores_soporte_clasificador_tf = GridSearchCV(estimator=pipeline_maquina_vectores_soporte_clasificador_tf, param_grid=param_grid_maquina_vectores_soporte_clasificador_tf, cv=2, n_jobs=1, verbose=6, scoring='accuracy')

In [ ]:
maquina_vectores_soporte_clasificador_tf_lematizacion = maquina_vectores_soporte_clasificador_tf

maquina_vectores_soporte_clasificador_tf_lematizacion.fit(X_train['tweet_tokenized_lematizacion_cleaned'].values, y_train)

y_pred_maquina_vectores_soporte_clasificador_tf_lematizacion = maquina_vectores_soporte_clasificador_tf_lematizacion.predict(X_test['tweet_tokenized_lematizacion_cleaned'].values)

Fitting 2 folds for each of 128 candidates, totalling 256 fits
[W] [19:41:56.392598] SVC with the linear kernel can be much faster using the specialized solver provided by LinearSVC. Consider switching to LinearSVC if tranining takes too long.
[CV 1/2] END clf__C=0.1, clf__gamma=scale, clf__kernel=linear, tfidf__max_df=0.75, tfidf__ngram_range=(1, 1);, score=0.498 total time=   3.0s
[CV 2/2] END clf__C=0.1, clf__gamma=scale, clf__kernel=linear, tfidf__max_df=0.75, tfidf__ngram_range=(1, 1);, score=0.490 total time=   0.2s
[CV 1/2] END clf__C=0.1, clf__gamma=scale, clf__kernel=linear, tfidf__max_df=0.75, tfidf__ngram_range=(1, 2);, score=0.370 total time=   0.2s
[CV 2/2] END clf__C=0.1, clf__gamma=scale, clf__kernel=linear, tfidf__max_df=0.75, tfidf__ngram_range=(1, 2);, score=0.368 total time=   0.2s
[CV 1/2] END clf__C=0.1, clf__gamma=scale, clf__kernel=linear, tfidf__max_df=1.0, tfidf__ngram_range=(1, 1);, score=0.498 total time=   0.1s
[CV 2/2] END clf__C=0.1, clf__gamma=scale, clf_

In [ ]:
joblib.dump(maquina_vectores_soporte_clasificador_tf_lematizacion, 'maquina_vectores_soporte_clasificador_tf_lematizacion.pkl')

['maquina_vectores_soporte_clasificador_tf_lematizacion.pkl']

##### **4.1.1.3. Regresión logística**

In [ ]:
pipeline_regresion_logistica_tf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(solver='qn'))
])

param_grid_regresion_logistica_tf = [
    {'tfidf__max_df': [0.75, 1.0],
     'tfidf__ngram_range': [(1, 1), (1, 2)],
     'clf__C': [0.01, 0.1, 1, 10, 100],
     'clf__penalty': ['l2'],
     'clf__max_iter': [100, 500, 1000, 2000],
     'clf__fit_intercept': [True, False]},
    {'tfidf__max_df': [0.75, 1.0],
     'tfidf__ngram_range': [(1, 1), (1, 2)],
     'clf__C': [0.01, 0.1, 1, 10, 100],
     'clf__penalty': ['l1'],
     'clf__max_iter': [100, 500, 1000, 2000],
     'clf__fit_intercept': [True, False]},
    {'tfidf__max_df': [0.75, 1.0],
     'tfidf__ngram_range': [(1, 1), (1, 2)],
     'clf__C': [0.01, 0.1, 1, 10, 100],
     'clf__penalty': ['elasticnet'],
     'clf__max_iter': [100, 500, 1000, 2000],
     'clf__fit_intercept': [True, False],
     'clf__l1_ratio': [0.5, 1]}
]

regresion_logistica_tf = GridSearchCV(estimator=pipeline_regresion_logistica_tf, param_grid=param_grid_regresion_logistica_tf, cv=5, n_jobs=-1, verbose=6, scoring='accuracy')

In [ ]:
regresion_logistica_tf_lematizacion = regresion_logistica_tf

regresion_logistica_tf_lematizacion.fit(X_train['tweet_tokenized_lematizacion_cleaned'].values, y_train)

y_pred_regresion_logistica_tf_lematizacion = regresion_logistica_tf_lematizacion.predict(X_test['tweet_tokenized_lematizacion_cleaned'].values)

Fitting 5 folds for each of 640 candidates, totalling 3200 fits


In [ ]:
joblib.dump(regresion_logistica_tf_lematizacion, 'regresion_logistica_tf_lematizacion.pkl')

['regresion_logistica_tf_lematizacion.pkl']

##### **4.1.1.4. Árboles de Decisión**

In [ ]:
pipeline_arbol_decision_tf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', DecisionTreeClassifier())
])

param_grid_arbol_decision_tf = {
    'tfidf__max_df': [0.75, 1.0],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__criterion': ['gini', 'entropy', 'log_loss'],
    'clf__splitter': ['best'],
    'clf__max_depth': [None, 10, 20, 30, 40, 50],
    'clf__min_samples_split': [2, 10, 20],
    'clf__min_samples_leaf': [1, 5, 10],
    'clf__max_features': [None, 'sqrt']
}

arbol_decision_tf = GridSearchCV(estimator=pipeline_arbol_decision_tf, param_grid=param_grid_arbol_decision_tf, cv=5, n_jobs=-1, verbose=6, scoring='accuracy')

In [ ]:
arbol_decision_tf_lematizacion = arbol_decision_tf

arbol_decision_tf_lematizacion.fit(X_train['tweet_tokenized_lematizacion_cleaned'].values, y_train)

y_pred_arbol_decision_tf_lematizacion = arbol_decision_tf_lematizacion.predict(X_test['tweet_tokenized_lematizacion_cleaned'].values)

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


In [ ]:
joblib.dump(arbol_decision_tf_lematizacion, 'arbol_decision_tf_lematizacion.pkl')

['arbol_decision_tf_lematizacion.pkl']

##### **4.1.1.5. Gradiente Descendente Estocástico**

In [ ]:
pipeline_sgd_tf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', SGDClassifier())
])

param_grid_sgd_tf = {
    'tfidf__max_df': [0.75, 1.0],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__loss': ['hinge', 'log_loss', 'modified_huber', 'squared_hinge', 'perceptron'],
    'clf__penalty': ['l2', 'l1', 'elasticnet'],
    'clf__alpha': [1e-4, 1e-3, 1e-2],
    'clf__max_iter': [1000, 2000],
    'clf__tol': [1e-3, 1e-4]
}

sgd_tf = GridSearchCV(estimator=pipeline_sgd_tf, param_grid=param_grid_sgd_tf, cv=5, n_jobs=-1, verbose=6, scoring='accuracy')

In [ ]:
sgd_tf_lematizacion = sgd_tf

sgd_tf_lematizacion.fit(X_train['tweet_tokenized_lematizacion_cleaned'].values, y_train)

y_pred_sgd_tf_lematizacion = sgd_tf_lematizacion.predict(X_test['tweet_tokenized_lematizacion_cleaned'].values)

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


In [ ]:
joblib.dump(sgd_tf_lematizacion, 'sgd_tf_lematizacion.pkl')

['sgd_tf_lematizacion.pkl']

##### **4.1.1.6. XGBoost**

In [ ]:
pipeline_xg_tf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
])

param_grid_xg_tf = {
    'tfidf__max_df': [0.75, 1.0],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__n_estimators': [100, 200],
    'clf__learning_rate': [0.01, 0.1],
    'clf__max_depth': [3, 5],
    'clf__subsample': [0.8, 0.9],
    'clf__colsample_bytree': [0.8, 0.9],
    'clf__gamma': [0, 0.1]
}

xg_tf = GridSearchCV(estimator=pipeline_xg_tf, param_grid=param_grid_xg_tf, cv=2, n_jobs=-1, verbose=6, scoring='accuracy')

In [ ]:
xg_tf_lematizacion = xg_tf

xg_tf_lematizacion.fit(X_train['tweet_tokenized_lematizacion_cleaned'].values, y_train)

y_pred_xg_tf_lematizacion = xg_tf_lematizacion.predict(X_test['tweet_tokenized_lematizacion_cleaned'].values)

Fitting 2 folds for each of 256 candidates, totalling 512 fits


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:09:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
joblib.dump(xg_tf_lematizacion, 'xg_tf_lematizacion.pkl')

['xg_tf_lematizacion.pkl']

##### **4.1.1.7. K Vecinos más cercanos**

In [ ]:
pipeline_knn_tf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', KNeighborsClassifier())
])

param_grid_knn_tf = {
    'tfidf__max_df': [0.75, 1.0],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__n_neighbors': [3, 5, 7, 9, 11, 30],
    'clf__weights': ['uniform', 'distance'],
    'clf__metric': ['euclidean', 'manhattan', 'minkowski']
}

knn_tf = GridSearchCV(estimator=pipeline_knn_tf, param_grid=param_grid_knn_tf, cv=5, n_jobs=-1, verbose=6, scoring='accuracy')

In [ ]:
knn_tf_lematizacion = knn_tf

knn_tf_lematizacion.fit(X_train['tweet_tokenized_lematizacion_cleaned'].values, y_train)

y_pred_knn_tf_lematizacion = knn_tf_lematizacion.predict(X_test['tweet_tokenized_lematizacion_cleaned'].values)

[I] [20:10:54.023637] Unused keyword parameter: n_jobs during cuML estimator initialization
Fitting 5 folds for each of 144 candidates, totalling 720 fits
[I] [20:10:54.028054] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:10:54.028776] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:10:54.029557] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:10:54.030957] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:10:54.031621] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:10:54.032400] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:10:54.033007] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:10:54.034500] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:10:54.035102] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:10:54.035

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[I] [20:11:04.078239] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:11:04.081146] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:11:04.081945] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:11:04.082785] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:11:04.083563] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:11:04.084312] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:11:04.085070] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:11:04.085841] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:11:04.086667] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:11:04.087658] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:11:04.088479] Unused keyword parameter: n_jobs during cuML estimator ini

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


[I] [20:11:12.256714] Unused keyword parameter: n_jobs during cuML estimator initialization


In [ ]:
joblib.dump(knn_tf_lematizacion, 'knn_tf_lematizacion.pkl')

['knn_tf_lematizacion.pkl']

##### **5.1.1.8. Bosques aleatorios**

In [ ]:
pipeline_rf_tf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', RandomForestClassifier())
])

param_grid_rf_tf = {
    'tfidf__max_df': [0.75, 1.0],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__n_estimators': [50, 100, 150],
    'clf__max_features': ['sqrt', 'log2'],
    'clf__min_samples_split': [2, 5, 10],
    'clf__criterion': ['gini', 'entropy']
}

rf_tf = GridSearchCV(estimator=pipeline_rf_tf, param_grid=param_grid_rf_tf, cv=7, n_jobs=-1, verbose=6, scoring='accuracy')

In [ ]:
rf_tf_lematizacion = rf_tf

rf_tf_lematizacion.fit(X_train['tweet_tokenized_lematizacion_cleaned'].values, y_train)

y_pred_rf_tf_lematizacion = rf_tf_lematizacion.predict(X_test['tweet_tokenized_lematizacion_cleaned'].values)

Fitting 7 folds for each of 144 candidates, totalling 1008 fits


In [ ]:
joblib.dump(rf_tf_lematizacion, 'rf_tf_lematizacion.pkl')

['rf_tf_lematizacion.pkl']

### **4.2. Con Tallado**

#### **4.2.1. TF-IDF**

##### **4.2.1.1. Naive Bayes Multinomial**

In [ ]:
naive_bayes_clasificador_tf_tallado = naive_bayes_clasificador_tf

naive_bayes_clasificador_tf_tallado.fit(X_train['tweet_tokenized_tallado_cleaned'].values, y_train)

y_pred_naive_bayes_clasificador_tf_tallado = naive_bayes_clasificador_tf_tallado.predict(X_test['tweet_tokenized_tallado_cleaned'].values)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


In [ ]:
joblib.dump(naive_bayes_clasificador_tf_tallado, 'naive_bayes_clasificador_tf_tallado.pkl')

['naive_bayes_clasificador_tf_tallado.pkl']

##### **4.2.1.2. Máquina de Vectores de Soporte**

In [ ]:
maquina_vectores_soporte_clasificador_tf_tallado = maquina_vectores_soporte_clasificador_tf

maquina_vectores_soporte_clasificador_tf_tallado.fit(X_train['tweet_tokenized_tallado_cleaned'].values, y_train)

y_pred_maquina_vectores_soporte_clasificador_tf_tallado = maquina_vectores_soporte_clasificador_tf_tallado.predict(X_test['tweet_tokenized_tallado_cleaned'].values)

Fitting 2 folds for each of 128 candidates, totalling 256 fits
[CV 1/2] END clf__C=0.1, clf__gamma=scale, clf__kernel=linear, tfidf__max_df=0.75, tfidf__ngram_range=(1, 1);, score=0.527 total time=   0.1s
[CV 2/2] END clf__C=0.1, clf__gamma=scale, clf__kernel=linear, tfidf__max_df=0.75, tfidf__ngram_range=(1, 1);, score=0.532 total time=   0.1s
[CV 1/2] END clf__C=0.1, clf__gamma=scale, clf__kernel=linear, tfidf__max_df=0.75, tfidf__ngram_range=(1, 2);, score=0.385 total time=   0.2s
[CV 2/2] END clf__C=0.1, clf__gamma=scale, clf__kernel=linear, tfidf__max_df=0.75, tfidf__ngram_range=(1, 2);, score=0.385 total time=   0.2s
[CV 1/2] END clf__C=0.1, clf__gamma=scale, clf__kernel=linear, tfidf__max_df=1.0, tfidf__ngram_range=(1, 1);, score=0.527 total time=   0.1s
[CV 2/2] END clf__C=0.1, clf__gamma=scale, clf__kernel=linear, tfidf__max_df=1.0, tfidf__ngram_range=(1, 1);, score=0.532 total time=   0.1s
[CV 1/2] END clf__C=0.1, clf__gamma=scale, clf__kernel=linear, tfidf__max_df=1.0, tfidf

In [ ]:
joblib.dump(maquina_vectores_soporte_clasificador_tf_tallado, 'maquina_vectores_soporte_clasificador_tf_tallado.pkl')

['maquina_vectores_soporte_clasificador_tf_tallado.pkl']

In [ ]:
maquina_vectores_soporte_clasificador_tf_tallado.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75, ngram_range=(1, 2))),
                ('clf', SVC())])

##### **4.2.1.3. Regresión logística**

In [ ]:
regresion_logistica_tf_tallado = regresion_logistica_tf

regresion_logistica_tf_tallado.fit(X_train['tweet_tokenized_tallado_cleaned'].values, y_train)

y_pred_regresion_logistica_tf_tallado = regresion_logistica_tf_tallado.predict(X_test['tweet_tokenized_tallado_cleaned'].values)

Fitting 5 folds for each of 640 candidates, totalling 3200 fits
[W] [20:33:23.799863] L-BFGS: max iterations reached
[W] [20:33:23.800297] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [20:33:23.799863] L-BFGS: max iterations reached
[W] [20:33:23.800297] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.


In [ ]:
joblib.dump(regresion_logistica_tf_tallado, 'regresion_logistica_tf_tallado.pkl')

['regresion_logistica_tf_tallado.pkl']

In [ ]:
regresion_logistica_tf_tallado.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75)),
                ('clf', LogisticRegression())])

##### **4.2.1.4. Árboles de Decisión**

In [ ]:
arbol_decision_tf_tallado = arbol_decision_tf

arbol_decision_tf_tallado.fit(X_train['tweet_tokenized_tallado_cleaned'].values, y_train)

y_pred_arbol_decision_tf_tallado = arbol_decision_tf_tallado.predict(X_test['tweet_tokenized_tallado_cleaned'].values)

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


In [ ]:
joblib.dump(arbol_decision_tf_tallado, 'arbol_decision_tf_tallado.pkl')

['arbol_decision_tf_tallado.pkl']

##### **4.2.1.5. Gradiente Descendente Estocástico**

In [ ]:
sgd_tf_tallado = sgd_tf

sgd_tf_tallado.fit(X_train['tweet_tokenized_tallado_cleaned'].values, y_train)

y_pred_sgd_tf_tallado = sgd_tf_tallado.predict(X_test['tweet_tokenized_tallado_cleaned'].values)

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [ ]:
joblib.dump(sgd_tf_tallado, 'sgd_tf_tallado.pkl')

['sgd_tf_tallado.pkl']

##### **4.2.1.6. XGBoost**

In [ ]:
xg_tf_tallado = xg_tf

xg_tf_tallado.fit(X_train['tweet_tokenized_tallado_cleaned'].values, y_train)

y_pred_xg_tf_tallado = xg_tf_tallado.predict(X_test['tweet_tokenized_tallado_cleaned'].values)

Fitting 2 folds for each of 256 candidates, totalling 512 fits


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:47:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
joblib.dump(xg_tf_tallado, 'xg_tf_tallado.pkl')

['xg_tf_tallado.pkl']

##### **4.2.1.7. K Vecinos más cercanos**

In [ ]:
knn_tf_tallado = knn_tf

knn_tf_tallado.fit(X_train['tweet_tokenized_tallado_cleaned'].values, y_train)

y_pred_knn_tf_tallado = knn_tf_tallado.predict(X_test['tweet_tokenized_tallado_cleaned'].values)

[I] [20:47:56.502030] Unused keyword parameter: n_jobs during cuML estimator initialization
Fitting 5 folds for each of 144 candidates, totalling 720 fits
[I] [20:47:56.505964] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:47:56.506671] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:47:56.507556] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:47:56.508213] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:47:56.508840] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:47:56.509488] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:47:56.510101] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:47:56.510745] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:47:56.511383] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:47:56.512

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[I] [20:48:09.701639] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:48:09.703233] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:48:09.704128] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:48:09.704993] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:48:09.705830] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:48:09.706698] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:48:09.708508] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:48:09.709241] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:48:09.709928] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:48:09.710696] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [20:48:09.711377] Unused keyword parameter: n_jobs during cuML estimator ini

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [ ]:
joblib.dump(knn_tf_tallado, 'knn_tf_tallado.pkl')

['knn_tf_tallado.pkl']

##### **4.2.1.8. Bosques aleatorios**

In [ ]:
rf_tf_tallado = rf_tf

rf_tf_tallado.fit(X_train['tweet_tokenized_tallado_cleaned'].values, y_train)

y_pred_rf_tf_tallado = rf_tf_tallado.predict(X_test['tweet_tokenized_tallado_cleaned'].values)

Fitting 7 folds for each of 144 candidates, totalling 1008 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [ ]:
rf_tf_tallado.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75)),
                ('clf',
                 RandomForestClassifier(criterion='entropy',
                                        max_features='log2',
                                        min_samples_split=5,
                                        n_estimators=150))])

In [ ]:
joblib.dump(rf_tf_tallado, 'rf_tf_tallado.pkl')

['rf_tf_tallado.pkl']

## **5. Evaluación de los modelos**

### **5.1. Con Lematización**

#### **5.1.1. TF-IDF**

##### **5.1.1.1. Naive Bayes Multinomial**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_naive_bayes_clasificador_tf_lematizacion.astype(int)))

              precision    recall  f1-score   support

           0       0.99      0.92      0.95       152
           1       0.96      0.96      0.96       131
           2       0.91      0.99      0.95       114
           3       0.98      0.98      0.98        83

    accuracy                           0.96       480
   macro avg       0.96      0.96      0.96       480
weighted avg       0.96      0.96      0.96       480



##### **5.1.1.2. Máquina de Vectores de Soporte**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_maquina_vectores_soporte_clasificador_tf_lematizacion.astype(int)))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93       152
           1       0.95      0.93      0.94       131
           2       0.93      0.99      0.96       114
           3       1.00      0.96      0.98        83

    accuracy                           0.95       480
   macro avg       0.95      0.95      0.95       480
weighted avg       0.95      0.95      0.95       480



##### **5.1.1.3. Regresión logística**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_regresion_logistica_tf_lematizacion.astype(int)))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95       152
           1       0.96      0.96      0.96       131
           2       0.96      1.00      0.98       114
           3       1.00      0.96      0.98        83

    accuracy                           0.96       480
   macro avg       0.97      0.97      0.97       480
weighted avg       0.96      0.96      0.96       480



##### **5.1.1.4. Árboles de Decisión**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_arbol_decision_tf_lematizacion.astype(int)))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95       152
           1       0.95      0.95      0.95       131
           2       0.99      0.96      0.98       114
           3       1.00      0.95      0.98        83

    accuracy                           0.96       480
   macro avg       0.97      0.96      0.96       480
weighted avg       0.96      0.96      0.96       480



##### **5.1.1.5. Gradiente Descendente Estocástico**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_sgd_tf_lematizacion.astype(int)))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94       152
           1       0.95      0.95      0.95       131
           2       0.96      0.99      0.97       114
           3       1.00      0.96      0.98        83

    accuracy                           0.96       480
   macro avg       0.96      0.96      0.96       480
weighted avg       0.96      0.96      0.96       480



##### **5.1.1.6. XGBoost**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_xg_tf_lematizacion.astype(int)))

              precision    recall  f1-score   support

           0       0.84      0.95      0.89       152
           1       0.91      0.88      0.89       131
           2       0.94      0.88      0.91       114
           3       0.96      0.89      0.93        83

    accuracy                           0.90       480
   macro avg       0.91      0.90      0.91       480
weighted avg       0.91      0.90      0.90       480



##### **5.1.1.7. K Vecinos más cercanos**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_knn_tf_lematizacion.astype(int)))

              precision    recall  f1-score   support

           0       0.84      0.85      0.84       152
           1       0.84      0.81      0.82       131
           2       0.78      0.82      0.80       114
           3       0.96      0.92      0.94        83

    accuracy                           0.84       480
   macro avg       0.85      0.85      0.85       480
weighted avg       0.85      0.84      0.84       480



##### **5.1.1.8. Bosques aleatorios**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_rf_tf_lematizacion.astype(int)))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95       152
           1       0.95      0.95      0.95       131
           2       0.99      0.98      0.99       114
           3       1.00      0.96      0.98        83

    accuracy                           0.96       480
   macro avg       0.97      0.96      0.97       480
weighted avg       0.97      0.96      0.96       480



### **5.2. Con Tallado**

#### **5.2.1. TF-IDF**

##### **5.2.1.1. Naive Bayes Multinomial**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_naive_bayes_clasificador_tf_tallado.astype(int)))

              precision    recall  f1-score   support

           0       0.99      0.91      0.95       152
           1       0.96      0.96      0.96       131
           2       0.89      0.99      0.94       114
           3       0.99      0.96      0.98        83

    accuracy                           0.95       480
   macro avg       0.96      0.96      0.96       480
weighted avg       0.96      0.95      0.95       480



##### **5.2.1.2. Máquina de Vectores de Soporte**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_maquina_vectores_soporte_clasificador_tf_tallado.astype(int)))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96       152
           1       0.96      0.95      0.96       131
           2       0.93      1.00      0.97       114
           3       1.00      0.96      0.98        83

    accuracy                           0.96       480
   macro avg       0.97      0.97      0.97       480
weighted avg       0.97      0.96      0.96       480



##### **5.2.1.3. Regresión logística**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_regresion_logistica_tf_tallado.astype(int)))

              precision    recall  f1-score   support

           0       0.97      0.94      0.95       152
           1       0.95      0.95      0.95       131
           2       0.94      1.00      0.97       114
           3       1.00      0.96      0.98        83

    accuracy                           0.96       480
   macro avg       0.97      0.96      0.96       480
weighted avg       0.96      0.96      0.96       480



##### **5.2.1.4. Árboles de Decisión**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_arbol_decision_tf_tallado.astype(int)))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       152
           1       1.00      0.95      0.98       131
           2       0.93      0.99      0.96       114
           3       1.00      0.92      0.96        83

    accuracy                           0.97       480
   macro avg       0.97      0.96      0.97       480
weighted avg       0.97      0.97      0.97       480



##### **5.2.1.5. Gradiente Descendente Estocástico**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_sgd_tf_tallado.astype(int)))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95       152
           1       0.97      0.94      0.95       131
           2       0.96      1.00      0.98       114
           3       1.00      0.96      0.98        83

    accuracy                           0.96       480
   macro avg       0.97      0.96      0.97       480
weighted avg       0.96      0.96      0.96       480



##### **5.2.1.6. XGBoost**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_xg_tf_tallado.astype(int)))

              precision    recall  f1-score   support

           0       0.88      0.96      0.92       152
           1       0.91      0.89      0.90       131
           2       0.94      0.90      0.92       114
           3       0.97      0.92      0.94        83

    accuracy                           0.92       480
   macro avg       0.93      0.92      0.92       480
weighted avg       0.92      0.92      0.92       480



##### **5.2.1.7. K Vecinos más cercanos**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_knn_tf_tallado.astype(int)))

              precision    recall  f1-score   support

           0       0.82      0.84      0.83       152
           1       0.82      0.85      0.84       131
           2       0.83      0.82      0.83       114
           3       0.97      0.89      0.93        83

    accuracy                           0.85       480
   macro avg       0.86      0.85      0.86       480
weighted avg       0.85      0.85      0.85       480



##### **5.2.1.8. Bosques aleatorios**

In [ ]:
print(metrics.classification_report(y_test.astype(int), y_pred_rf_tf_tallado.astype(int)))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96       152
           1       0.95      0.95      0.95       131
           2       0.97      0.97      0.97       114
           3       1.00      0.96      0.98        83

    accuracy                           0.96       480
   macro avg       0.97      0.96      0.97       480
weighted avg       0.96      0.96      0.96       480

